In [20]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from calendar import monthrange
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np


In [21]:
# Masquer les SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [22]:
df = pd.read_csv("https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_03.csv")

In [23]:
df_stations_old = df.groupby(["NUM_POSTE"]).max()
print(len(df_stations_old))

1129


In [24]:
# filter out station qui n'ont pas de données récentes (2022)
last_year = df["Year"].max()
mask = df_stations_old["Year"]!=last_year
df_stations_old = df_stations_old[mask]
stations_old = df_stations_old.index.to_list()
df_recent = df[~df["NUM_POSTE"].isin(stations_old)]

In [25]:
print(len(df_recent.groupby(["NUM_POSTE"]).max()))

737


In [26]:
# check continuité des données sur 7 ans
# UPDATE : le modèle ne peut pas prédire avec moins de 8 années de training

stations = df_recent["NUM_POSTE"].unique()
to_drop = []

for num_station in stations:
    df_station = df_recent[df_recent["NUM_POSTE"]==num_station]

    for year in range(last_year,last_year-8,-1):
        nb_missing_month = 0
        try:
            df_station_year = df_station[df_station["Year"]==year]
            nb_missing_month = 12 - len(df_station_year["Month"].unique().tolist())
            if nb_missing_month > 0:
                to_drop.append(num_station)
                print(f"Drop station N°{num_station}")
                print(f"{nb_missing_month} manquants en {year}\n\n")
            else:
                print(f"Station N°{num_station} has {df_station_year["Month"].count()} months in {year}")

        except Exception as e:
            to_drop.append(num_station)
            print(e)
            print()
            print(f"Drop station N°{num_station}")
            print(f"{year} est manquante\n\n")

    print()

Station N°1014002 has 12 months in 2022
Station N°1014002 has 12 months in 2021
Station N°1014002 has 12 months in 2020
Station N°1014002 has 12 months in 2019
Station N°1014002 has 12 months in 2018
Station N°1014002 has 12 months in 2017
Station N°1014002 has 12 months in 2016
Station N°1014002 has 12 months in 2015

Drop station N°1033002
1 manquants en 2022


Station N°1033002 has 12 months in 2021
Station N°1033002 has 12 months in 2020
Station N°1033002 has 12 months in 2019
Station N°1033002 has 12 months in 2018
Station N°1033002 has 12 months in 2017
Station N°1033002 has 12 months in 2016
Station N°1033002 has 12 months in 2015

Station N°1034004 has 12 months in 2022
Station N°1034004 has 12 months in 2021
Station N°1034004 has 12 months in 2020
Station N°1034004 has 12 months in 2019
Station N°1034004 has 12 months in 2018
Station N°1034004 has 12 months in 2017
Station N°1034004 has 12 months in 2016
Station N°1034004 has 12 months in 2015

Station N°1071001 has 12 months 

In [27]:
df_recent = df_recent[~df_recent["NUM_POSTE"].isin(to_drop)]

In [28]:
print(len(df_recent.groupby(["NUM_POSTE"]).max()))

648


In [29]:
# PREPROCESSING DATE

df_recent["AAAAMM"] = pd.to_datetime(df_recent["AAAAMM"])

def last_day_of_month(aaaamm):
    """
    prend une date format AAAAMM et retourne une string format AAAA-MM-DD avec DD = dernier jour du mois
    """
    year = aaaamm.year
    month = aaaamm.month
    last_day = monthrange(year, month)[1]
    return f"{year}-{month:02d}-{last_day:02d}"

df_recent["date"] = df_recent["AAAAMM"].apply(last_day_of_month)
df_recent["date"] = pd.to_datetime(df_recent["date"])


In [30]:
to_keep = ["NUM_POSTE","NOM_USUEL","LAT","LON","ALTI","Year","Month","vent_speed_inst_moy_mensu","departement_num","departement_name","region","date"]
to_drop = [col for col in df if col not in to_keep]

In [31]:
df_recent = df_recent.drop(to_drop,axis=1)

In [32]:
def clean_continuity(dataframe):

    """
    INPUT = un dataframe, le nom de sa variable date
    --> parcours la série d'année en année en commençant par la plus récente
    --> si année en cours < 12 mois, drop l'année et toutes les années antérieures
    --> si "trou" entre année en cours et année suivante, drop N-1 et toutes les années antérieures 
    --> transforme en série temporelle à frequence mensuelle
    
    OUTPUT = série temporelle clean
    """

    print(f"Cleaning de la station {dataframe["NUM_POSTE"].unique().tolist()} ...")

    #--> check continuité des années : stop_year = N si N < 12 mois, N-1 si "trou" entre N et N-1
    years= df_station["Year"].unique().tolist()
    years.sort(reverse=True)

    month_unique = []
    stop_year = 0

    for index, year in enumerate(years):
        df_station_year = df_station[df_station["Year"] == year]
        month_unique = df_station_year["Month"].unique().tolist()
        if len(month_unique)!=12:
            stop_year = year
            print(f"Année : {year}. Continuité rompue.")
            print(f"Mois: {len(month_unique)} / 12.")
            break
        else:
            try:
                if year - years[index+1] != 1:
                    stop_year = years[index+1]
                    print(f"Année : {years[index+1]}. Continuité rompue.")
                    print(f"Année manquante : {years[index+1]}")
                    break
            except IndexError:
                print("Continuité garantie")
                pass

    print("...Terminé.")

    #--> drop
    if stop_year > 0:
        dataframe = dataframe[dataframe["Year"] > stop_year]

        print(f"Données antérieures à {stop_year+1} supprimées.")
    else:
        print(f"Aucune rupture de continuité constatée, toutes les données ont été conservées.")
    

    #--> transforme en série temporelle à frequence mensuelle
    dataframe.set_index("date", inplace=True)
    dataframe = dataframe.asfreq("ME")

    #--> check NA
    check = "NOT OK" if dataframe.isna().any().any() else "OK"
    print(f"Check valeurs manquantes : {check}\n\n")

    return dataframe
    

In [33]:
def get_wind_forecast(time, data):

    """
    INPUT =  nombre de mois (time) à prédire et le set de données

    --> prédit la force du vent à horizon "time"
    
    OUTPUT =  le df de test, les prédictions, la MAE et la MAPE
    """
    # calcule date de split 
    date_limite = data.index.max() - pd.DateOffset(months=time)

    # split train / test
    train = data[data.index <= date_limite]
    test = data[data.index > date_limite]

    # split variables exogènes / target
    to_keep = ["LON","LAT","ALTI"]

    var_exog_train = train[to_keep]
    y_train = train["vent_speed_inst_moy_mensu"]

    var_exog_test = test[to_keep]
    y_test = test["vent_speed_inst_moy_mensu"]

    # training
    model = SARIMAX(y_train, exog=var_exog_train, order=(1, 1, 1), seasonal_order=(2, 1, 2, 12))
    sarima_model = model.fit(maxiter=1000)

    # prédictions
    forecast = sarima_model.get_forecast(steps = time, exog = var_exog_test)
    predicted_values = forecast.predicted_mean

    # évaluation
    mae = mean_absolute_error(y_test, predicted_values)
    mape = mean_absolute_percentage_error(y_test, predicted_values)

    # rendu
    return test, predicted_values, mae, mape


In [34]:
stations

array([ 1014002,  1033002,  1034004,  1071001,  1089001,  1414001,
        2037002,  2094001,  2110002,  2173002,  2320001,  2321002,
        2705001,  3060001,  3155003,  3180001,  3185007,  3248001,
        4019001,  4049001,  4068001,  4070009,  4096401,  4134002,
        4136001,  4209005,  4230001,  5007003,  5044400,  5046001,
        5055001,  5061009,  5063402,  5070003,  5077402,  5079402,
        5096404,  5120002,  5126001,  5136002,  5145002,  5170001,
        5181002,  5183001,  6004002,  6005001,  6023004,  6029001,
        6037002,  6038001,  6075007,  6077006,  6079002,  6081001,
        6083005,  6088001,  6088007,  6090002,  6091003,  6094002,
        6118002,  6136005,  6152002,  6153400,  7025001,  7032002,
        7068001,  7096001,  7131001,  7154005,  7172002,  7187001,
        7202002,  7204008,  8105005,  8145001,  8353001,  8367002,
        8401001,  9024004,  9099001,  9161003,  9199002,  9285400,
        9289001, 10030001, 10057001, 10070001, 10228002, 10350

In [35]:
# créé un dictionnaire contenant un dataframe par numéro de station
# clean continuité des dates sur le dataframe + transforme en série temporelle (clean_continuity)
# prédit (get_wind_forecast) et enregistre les prédictions horizon, la MAE et la MAPE générale
# enregistre le dataframe clean et le dataframe d'évaluation des prédictions

stations = df_recent["NUM_POSTE"].unique()
horizon = 84

dict_stations = {}

for station in stations:
    
    df_station = df_recent[df_recent["NUM_POSTE"]==station]
    df_station = clean_continuity(df_station)

    df_station_eval, predictions, mae, mape = get_wind_forecast(horizon,df_station)
    df_station_eval[f"pred_{horizon}_mois"] = predictions
    df_station_eval["mae"] = mae
    df_station_eval["mape"] = mape

    dict_stations[station] = [df_station,df_station_eval]




Cleaning de la station [1014002] ...
Année : 2004. Continuité rompue.
Mois: 10 / 12.
...Terminé.
Données antérieures à 2005 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [1034004] ...
Année : 2001. Continuité rompue.
Mois: 3 / 12.
...Terminé.
Données antérieures à 2002 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [1071001] ...
Année : 2002. Continuité rompue.
Mois: 8 / 12.
...Terminé.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [1089001] ...
Année : 1991. Continuité rompue.
Mois: 11 / 12.
...Terminé.
Données antérieures à 1992 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [1414001] ...
Année : 2002. Continuité rompue.
Mois: 4 / 12.
...Terminé.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [2037002] ...
Continuité garantie
...Terminé.
Aucune rupture de continuité constatée, toutes les données ont été conservées.
Check valeurs m

c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


Cleaning de la station [3155003] ...
Année : 1991. Continuité rompue.
Mois: 11 / 12.
...Terminé.
Données antérieures à 1992 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [3180001] ...
Année : 1991. Continuité rompue.
Mois: 7 / 12.
...Terminé.
Données antérieures à 1992 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [3185007] ...
Année : 2002. Continuité rompue.
Mois: 9 / 12.
...Terminé.
Données antérieures à 2003 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [3248001] ...
Année : 1991. Continuité rompue.
Mois: 10 / 12.
...Terminé.
Données antérieures à 1992 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [4019001] ...
Année : 1998. Continuité rompue.
Mois: 11 / 12.
...Terminé.
Données antérieures à 1999 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [4049001] ...
Année : 1990. Continuité rompue.
Mois: 11 / 12.
...Terminé.
Données antérieures à 1991 supprimées.
Check valeurs manquantes :

c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


Cleaning de la station [4136001] ...
Année : 2009. Continuité rompue.
Mois: 1 / 12.
...Terminé.
Données antérieures à 2010 supprimées.
Check valeurs manquantes : OK




...

Cleaning de la station [91027002] ...
Continuité garantie
...Terminé.
Aucune rupture de continuité constatée, toutes les données ont été conservées.
Check valeurs manquantes : OK


Cleaning de la station [91184001] ...
Année : 1998. Continuité rompue.
Mois: 3 / 12.
...Terminé.
Données antérieures à 1999 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [91200002] ...
Année : 2006. Continuité rompue.
Mois: 7 / 12.
...Terminé.
Données antérieures à 2007 supprimées.
Check valeurs manquantes : OK




c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


Cleaning de la station [95078001] ...
Année : 1993. Continuité rompue.
Mois: 10 / 12.
...Terminé.
Données antérieures à 1994 supprimées.
Check valeurs manquantes : OK


Cleaning de la station [95088001] ...
Année : 2010. Continuité rompue.
Mois: 6 / 12.
...Terminé.
Données antérieures à 2011 supprimées.
Check valeurs manquantes : OK




c:\Users\alexm\anaconda3\envs\jedha_project\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


Cleaning de la station [95527001] ...
Continuité garantie
...Terminé.
Aucune rupture de continuité constatée, toutes les données ont été conservées.
Check valeurs manquantes : OK




In [36]:
# 2ème station de la liste, dataframe initial
dict_stations[stations[1]][0]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region
date,,,,,,,,,,,
2002-01-31,1034004,BELLEY,45.769333,5.688,330,2002,1,6.1,1,Ain,AUVERGNE RHONE ALPES
2002-02-28,1034004,BELLEY,45.769333,5.688,330,2002,2,5.4,1,Ain,AUVERGNE RHONE ALPES
2002-03-31,1034004,BELLEY,45.769333,5.688,330,2002,3,7.2,1,Ain,AUVERGNE RHONE ALPES
2002-04-30,1034004,BELLEY,45.769333,5.688,330,2002,4,9.0,1,Ain,AUVERGNE RHONE ALPES
2002-05-31,1034004,BELLEY,45.769333,5.688,330,2002,5,6.5,1,Ain,AUVERGNE RHONE ALPES
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,1034004,BELLEY,45.769333,5.688,330,2022,8,6.8,1,Ain,AUVERGNE RHONE ALPES
2022-09-30,1034004,BELLEY,45.769333,5.688,330,2022,9,6.1,1,Ain,AUVERGNE RHONE ALPES
2022-10-31,1034004,BELLEY,45.769333,5.688,330,2022,10,4.7,1,Ain,AUVERGNE RHONE ALPES


In [37]:
# 2ème station de la liste, dataframe d'évaluation des prédictions
dict_stations[stations[1]][1]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,Year,Month,vent_speed_inst_moy_mensu,departement_num,departement_name,region,pred_84_mois,mae,mape
date,,,,,,,,,,,,,,
2016-01-31,1034004,BELLEY,45.769333,5.688,330,2016,1,5.4,1,Ain,AUVERGNE RHONE ALPES,5.007070,0.69152,0.105778
2016-02-29,1034004,BELLEY,45.769333,5.688,330,2016,2,7.6,1,Ain,AUVERGNE RHONE ALPES,5.862264,0.69152,0.105778
2016-03-31,1034004,BELLEY,45.769333,5.688,330,2016,3,7.2,1,Ain,AUVERGNE RHONE ALPES,7.088174,0.69152,0.105778
2016-04-30,1034004,BELLEY,45.769333,5.688,330,2016,4,7.6,1,Ain,AUVERGNE RHONE ALPES,7.273306,0.69152,0.105778
2016-05-31,1034004,BELLEY,45.769333,5.688,330,2016,5,7.6,1,Ain,AUVERGNE RHONE ALPES,6.788167,0.69152,0.105778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,1034004,BELLEY,45.769333,5.688,330,2022,8,6.8,1,Ain,AUVERGNE RHONE ALPES,5.405402,0.69152,0.105778
2022-09-30,1034004,BELLEY,45.769333,5.688,330,2022,9,6.1,1,Ain,AUVERGNE RHONE ALPES,5.527201,0.69152,0.105778
2022-10-31,1034004,BELLEY,45.769333,5.688,330,2022,10,4.7,1,Ain,AUVERGNE RHONE ALPES,4.901957,0.69152,0.105778


In [38]:
# construction du dataframe de comparaison inter station

num_stations = []
nb_month_available = []
lat = []
lon = []
alti = []
true_val = []
pred_val = []
mae = []
mape = []

for key, value in dict_stations.items():
    num_stations.append(key)
    nb_month_available.append(len(value[0]))
    date_pred = value[1].index.max()

    data = value[1][value[1].index==date_pred]

    lat.append(data.loc[date_pred,"LAT"])
    lon.append(data.loc[date_pred,"LON"])
    alti.append(data.loc[date_pred,"ALTI"])
    true_val.append(data.loc[date_pred,"vent_speed_inst_moy_mensu"])
    pred_val.append(data.loc[date_pred,f"pred_{horizon}_mois"])
    mae.append(data.loc[date_pred,"mae"])
    mape.append(data.loc[date_pred,"mape"])

data = {
    "Num_station": num_stations,
    "Nb_mois_dispo": nb_month_available,
    "Latitude": lat,
    "Longitude": lon,
    "Altitude": alti,
    "Reel_2022-12": true_val,
    "Pred_2022-12": pred_val,
    "MAE": mae,
    "MAPE": mape,
}

df_eval_global = pd.DataFrame(data)


In [40]:
df_eval_global.describe()

,Num_station,Nb_mois_dispo,Latitude,Longitude,Altitude,Reel_2022-12,Pred_2022-12,MAE,MAPE
count,6.480000e+02,648.000000,648.000000,648.000000,648.000000,648.000000,648.000000,648.000000,648.000000
mean,4.337910e+07,327.685185,46.000632,3.047563,372.450617,11.581481,13.103281,2.408903,0.200621
std,2.692273e+07,209.463109,2.280524,3.108085,472.548319,4.979944,22.881347,8.974568,0.449023
min,1.014002e+06,96.000000,41.374833,-5.057000,1.000000,1.800000,-58.980740,0.390926,0.068212
25%,2.009275e+07,192.000000,43.927459,0.884625,86.750000,7.900000,8.058123,1.136245,0.110195
50%,4.005550e+07,246.000000,45.723917,3.140500,200.500000,11.200000,11.286838,1.384647,0.129605
75%,6.702750e+07,384.000000,48.039708,5.597542,458.500000,14.400000,14.885236,1.865622,0.158573
max,9.552700e+07,876.000000,51.055833,9.541500,3093.000000,36.700000,539.266084,212.837729,7.735205
